# <center> <font face=timenewroman color=red size=10> Cancer diagnosis with Deep Learning</center>


# <center> <font face=timenewroman color=yellow> Mount drive and Management files </center>

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
cd /content/drive/MyDrive/ChestCT-Classifier

/content/drive/MyDrive/ChestCT-Classifier


In [6]:
!find '.' -name '*.ipynb_checkpoints' -exec rm -r {} +

In [7]:
#Dataset

Dataset = '/content/drive/MyDrive/ChestCT-Classifier/dataset_lung'

#Model
model_path = '/content/drive/MyDrive/ChestCT-Classifier/models/'

# <center> <font face=timenewroman color=yellow>  Import modules </center>

## <center> <font face=timenewroman color=gree>Install timm (Pretrained model module)</center>

In [8]:
%%capture
!pip install timm

## <center> <font face=timenewroman color=gree>Import modules</center>

In [9]:
import os
from PIL import Image
import timm
from torchvision import datasets
import torchvision
import torch
from torchvision.transforms import transforms
import numpy as np
from PIL import ImageFile
import matplotlib.pyplot as plt
import json
import warnings
from sklearn.metrics import confusion_matrix as cm
import cv2
from IPython.display import clear_output
import time
import glob
import shutil

warnings.filterwarnings("ignore")
ImageFile.LOAD_TRUNCATED_IMAGES = True

# <center> <font face=timenewroman color=yellow>Prepare dataset</center>

## <center> <font face=timenewroman color=gree>Input parameters</center>

In [10]:
batch_size = 32
num_workers = 0
shuffle = True

#Peprocessing
transform = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])])

## <center> <font face=timenewroman color=gree>Dataset</center>

In [11]:
train_set = Dataset + '/train'
valid_set = Dataset + '/valid'

train_data = datasets.ImageFolder(train_set, transform=transform)
valid_data = datasets.ImageFolder(valid_set, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers, shuffle=shuffle)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, num_workers=num_workers, shuffle=shuffle)

loaders = {
    'train': train_loader,
    'valid': valid_loader
}

# <center> <font face=timenewroman color=yellow>Utils</center>

## <center> <font face=timenewroman color=gree>Criterion</center>

In [12]:
def cma(cm):
  """
  Confusion Matrix Analyse
  """
  recall    = []
  precision = []
  f1        = []
  for i in range(cm.shape[0]):
    recall.append(cm[i][i]/cm.sum(axis=0)[i])
    precision.append(cm[i][i]/cm.sum(axis=1)[i])
    f1.append(2 * (((cm[i][i]/cm.sum(axis=0)[i]) * (cm[i][i]/cm.sum(axis=1)[i]))/((cm[i][i]/cm.sum(axis=0)[i]) + (cm[i][i]/cm.sum(axis=1)[i]))))

  recall    = [x for x in recall if str(x) != 'nan']
  precision = [x for x in precision if str(x) != 'nan']
  f1        = [x for x in f1 if str(x) != 'nan']

  return [sum(recall)/len(recall), sum(precision)/len(precision), sum(f1)/len(f1)]

## <center> <font face=timenewroman color=gree>Train</center>

In [13]:
def train(n_epochs, loader, model, optimizer, criterion, use_cuda, save_path, model_name):
    train_accuracy_list = []
    train_loss_list = []

    train_recall_list = []
    train_precision_list = []
    train_f1_list = []

    valid_accuracy_list = []
    valid_loss_list = []

    valid_recall_list = []
    valid_precision_list = []
    valid_f1_list = []

    valid_acc_max = 0.0
    valid_r_max = 0.0
    valid_p_max = 0.0
    valid_f1_max = 0.0

    train_Confusion_matrix = np.zeros((4, 4))
    valid_Confusion_matrix = np.zeros((4, 4))

    for epoch in range(1, (n_epochs+1)):

        train_loss = 0.0
        valid_loss = 0.0
        train_acc = 0.0
        valid_acc = 0.0

        model.train()
        start = time.time()
        for batch_idx, (data, target) in enumerate(loader['train']):
            target_1 = target
            if use_cuda:
                data, target_2 = data.cuda(), target.cuda()

            optimizer.zero_grad()
            output = model(data)
            _, preds = torch.max(output, 1)
            loss = criterion(output, target_2)
            loss.backward()
            optimizer.step()
            train_Confusion_matrix += cm(preds.cpu().detach().numpy(), target_1.numpy(), labels=[i for i in range(4)])
            train_acc = train_acc + torch.sum(preds == target_2.data)
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))


        model.eval()
        for batch_idx, (data, target) in enumerate(loader['valid']):
            target_1 = target
            if use_cuda:
                data, target_2 = data.cuda(), target.cuda()
            output = model(data)

            _, preds = torch.max(output, 1)
            loss = criterion(output, target_2)

            valid_Confusion_matrix += cm(preds.cpu().detach().numpy(), target_1, labels=[i for i in range(4)])
            valid_acc = valid_acc + torch.sum(preds == target_2.data)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))


        train_loss = train_loss/len(loader['train'].dataset)
        valid_loss = valid_loss/len(loader['valid'].dataset)
        train_acc = train_acc/len(loader['train'].dataset)
        valid_acc = valid_acc/len(loader['valid'].dataset)


        train_result = cma(train_Confusion_matrix)
        valid_result = cma(valid_Confusion_matrix)

        train_recall_list.append(train_result[0])
        train_precision_list.append(train_result[1])
        train_f1_list.append(train_result[2])

        valid_recall_list.append(valid_result[0])
        valid_precision_list.append(valid_result[1])
        valid_f1_list.append(valid_result[2])

        train_loss_list.append(train_loss)
        valid_loss_list.append(valid_loss)

        train_accuracy_list.append(train_acc)
        valid_accuracy_list.append(valid_acc)


        Epoch_time = time.time() - start
        print('Epoch: {}\
              \n Epoch Time: {:6f}\
              \nTraining Acc: {:6f}\
              \nTraining Loss: {:6f}\
              \nTraining recall: {:6f}\
              \nTraining precision: {:6f}\
              \nTraining F1 Score: {:6f}\
              \n\n\
              \nValidation Acc: {:6f}\
              \nValidation Loss: {:.6f}\
              \nValidation recall: {:6f}\
              \nValidation precision: {:6f}\
              \nValidation F1 Score: {:6f}           '.format(
                                                        epoch,
                                                        Epoch_time,
                                                        train_acc,
                                                        train_loss,
                                                        train_result[0],
                                                        train_result[1],
                                                        train_result[2],
                                                        valid_acc,
                                                        valid_loss,
                                                        valid_result[0],
                                                        valid_result[1],
                                                        valid_result[2]
                                                        ))

        if valid_acc >= valid_acc_max:
            print('\n\nValidation accuracy increased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_acc_max,
            valid_acc))
            torch.save(model.state_dict(), save_path + '/model_' + model_name + '_acc_max.pt')
            valid_acc_max = valid_acc

        if valid_result[0] >= valid_r_max:
            print('\n\nValidation recall true increased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_r_max,
            valid_result[0]))
            torch.save(model.state_dict(), save_path + '/model_' + model_name + '_r_max.pt')
            valid_r_max = valid_result[0]

        if valid_result[1] >= valid_p_max:
            print('\n\nValidation precision true increased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_p_max,
            valid_result[1]))
            torch.save(model.state_dict(), save_path + '/model_' + model_name + '_p_max.pt')
            valid_p_max = valid_result[1]

        if valid_result[2] >= valid_f1_max:
            print('\n\nValidation F1 true increased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_f1_max,
            valid_result[2]))
            torch.save(model.state_dict(), save_path + '/model_' + model_name + '_f1_max.pt')
            valid_f1_max = valid_result[2]


        print('\n\n ------------------------------------------------------- \n\n')


    train_loss_list     = [i.cpu().tolist() for i in train_loss_list]
    valid_loss_list     = [i.cpu().tolist() for i in valid_loss_list]
    train_accuracy_list = [i.cpu().tolist() for i in train_accuracy_list]
    valid_accuracy_list = [i.cpu().tolist() for i in valid_accuracy_list]

    with open(save_path + '/model_' + model_name + '_train_accuracy_list.json', 'w') as f:
      json.dump(train_accuracy_list, f, indent=2)

    with open(save_path + '/model_' + model_name + '_train_loss_list.json', 'w') as f:
      json.dump(train_loss_list, f, indent=2)

    with open(save_path + '/model_' + model_name + '_train_recall_list.json', 'w') as f:
      json.dump(train_recall_list, f, indent=2)

    with open(save_path + '/model_' + model_name + '_train_precision_list.json', 'w') as f:
      json.dump(train_precision_list, f, indent=2)

    with open(save_path + '/model_' + model_name + '_train_f1_list.json', 'w') as f:
      json.dump(train_f1_list, f, indent=2)

    with open(save_path + '/model_' + model_name + '_valid_accuracy_list.json', 'w') as f:
      json.dump(valid_accuracy_list, f, indent=2)

    with open(save_path + '/model_' + model_name + '_valid_loss_list.json', 'w') as f:
      json.dump(valid_loss_list, f, indent=2)

    with open(save_path + '/model_' + model_name + '_valid_recall_list.json', 'w') as f:
      json.dump(valid_recall_list, f, indent=2)

    with open(save_path + '/model_' + model_name + '_valid_precision_list.json', 'w') as f:
      json.dump(valid_precision_list, f, indent=2)

    with open(save_path + '/model_' + model_name + '_valid_f1_list.json', 'w') as f:
      json.dump(valid_f1_list, f, indent=2)

    return model


## <center> <font face=timenewroman color=gree>Predict</center>

In [14]:
def predict(image, model, device, class_name):

    prediction_transform = transforms.Compose([transforms.Resize(size=(224, 224)),
                                               transforms.ToTensor(),
                                               transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    try:
      image = prediction_transform(image)[:3,:,:].unsqueeze(0)
    except:
      image = image.convert('RGB')
      image = prediction_transform(image)[:3,:,:].unsqueeze(0)

    if device == 'cuda':
        if torch.cuda.is_available():
            image = image.cuda()
        else:
            print("You don't have cuda")

    with torch.no_grad():
      model.eval()
      pred = model(image)


    idx = torch.argmax(pred)

    prob = pred[0][idx].item()*100

    return prob, class_name[idx]

## <center> <font face=timenewroman color=gree>Single frame app</center>

In [15]:
def app(path, model, device, class_name, display_prob, show):
    img = Image.open(path)
    if show:
      plt.imshow(img)
      plt.show()

    prob, result = predict(img, model, device, class_name)
    if display_prob:
      print('Probability of {} : {:.6f}'.format(result, prob))

    return prob, result


# <center> <font face=timenewroman color=yellow>Train</center>

## <center> <font face=timenewroman color=gree>Prepare model</center>

<font face=timenewroman color=white size=5>Pretrain Models in Timm

In [16]:
timm.list_models()

['aimv2_1b_patch14_224',
 'aimv2_1b_patch14_336',
 'aimv2_1b_patch14_448',
 'aimv2_3b_patch14_224',
 'aimv2_3b_patch14_336',
 'aimv2_3b_patch14_448',
 'aimv2_huge_patch14_224',
 'aimv2_huge_patch14_336',
 'aimv2_huge_patch14_448',
 'aimv2_large_patch14_224',
 'aimv2_large_patch14_336',
 'aimv2_large_patch14_448',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'beitv2_base_patch16_224',
 'beitv2_large_patch16_224',
 'botnet26t_256',
 'botnet50ts_256',
 'caformer_b36',
 'caformer_m36',
 'caformer_s18',
 'caformer_s36',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_medium',
 'coat_lite_medium_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_small',
 'coat_tiny',
 'coatnet_0_224',
 'coatnet_0_rw_224',
 'coa

<font face=timenewroman color=white size=5>We are using the ResNet50 model

In [17]:
model = timm.create_model('vgg16', pretrained=True,  num_classes=3)

use_cuda = torch.cuda.is_available()

if use_cuda:
    model = model.cuda()

model.safetensors:   0%|          | 0.00/553M [00:00<?, ?B/s]

<font face=timenewroman color=white size=5>Models structure

In [18]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [19]:


def unfreeze_resnet(model):
  for param in model.parameters():
    param.requires_grad = False
  model.fc = torch.nn.Sequential(torch.nn.Linear(2048, 256),
                                        torch.nn.Dropout(0.2),
                                        torch.nn.ReLU(),
                                        torch.nn.Linear(256, 64),
                                        torch.nn.Dropout(0.2),
                                        torch.nn.ReLU(),
                                        torch.nn.Linear(64, 32),
                                        torch.nn.Dropout(0.2),
                                        torch.nn.ReLU(),
                                        torch.nn.Linear(32, 3),
                                        torch.nn.Softmax()
                                        )

  for param in model.fc.parameters():
      param.requires_grad = True
  return model

def unfreeze_vgg(model):
  # Freeze all layers except the final fully connected layer
  for param in model.parameters():
    param.requires_grad = False

# Unfreeze the last fully connected layer for training
  for param in model.head.parameters():
    param.requires_grad = True
  return model

unfreeze_vgg(model)

if use_cuda:
    model_transfer = model.cuda()
else:
    model_transfer = model

## <center> <font face=timenewroman color=gree>Start Train</center>

### <font face=timenewroman color=blue size=5> Parameters

In [20]:
n_epochs         = 20
lr               = 0.001
wd               = 0.0001
model_name       = 'vgg16'
save_model_path  = os.path.join(model_path, model_name)
use_cuda         = True

In [21]:
save_model_path

'/content/drive/MyDrive/ChestCT-Classifier/models/vgg16'

### <font face=timenewroman color=blue size=5>Train model

In [22]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model_transfer.head.parameters(), lr=lr, weight_decay=wd)

In [23]:
model = train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_model_path, model_name)

Epoch: 1              
 Epoch Time: 418.093239              
Training Acc: 0.803409              
Training Loss: 0.000583              
Training recall: 0.609779              
Training precision: 0.701909              
Training F1 Score: 0.604186              

              
Validation Acc: 0.838710              
Validation Loss: 0.001744              
Validation recall: 0.712846              
Validation precision: 0.764302              
Validation F1 Score: 0.715032           


Validation accuracy increased (0.000000 --> 0.838710).  Saving model ...


Validation recall true increased (0.000000 --> 0.712846).  Saving model ...


Validation precision true increased (0.000000 --> 0.764302).  Saving model ...


Validation F1 true increased (0.000000 --> 0.715032).  Saving model ...


 ------------------------------------------------------- 


Epoch: 2              
 Epoch Time: 24.765657              
Training Acc: 0.895455              
Training Loss: 0.000315              
Training re

### <font face=timenewroman color=blue size=5>Result

In [24]:
loss_t = open(os.path.join(save_model_path, 'model_ResNet50_train_loss_list.json'))
loss_t = json.load(loss_t)

loss_v = open(os.path.join(save_model_path, 'model_ResNet50_valid_loss_list.json'))
loss_v = json.load(loss_v)

plt.ion()
plt.style.use("ggplot")
plt.figure()
plt.plot(loss_t, label="train_loss")
plt.plot(loss_v, label="valid_loss")
plt.title("Compare Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="upper right")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/ChestCT-Classifier/models/vgg16/model_ResNet50_train_loss_list.json'

In [ ]:
acc_t = open(os.path.join(model_path, 'model_ResNet50_train_accuracy_list.json'))
acc_t = json.load(acc_t)

acc_v = open(os.path.join(model_path, 'model_ResNet50_valid_accuracy_list.json'))
acc_v = json.load(acc_v)

plt.ion()
plt.style.use("ggplot")
plt.figure()

plt.plot(acc_t, label="train_acc")
plt.plot(acc_v, label="valid_acc")

plt.title("Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend(loc="lower right")

In [ ]:
rec_t = open(os.path.join(model_path, 'model_ResNet50_train_recall_list.json'))
rec_t = json.load(rec_t)

rec_v = open(os.path.join(model_path, 'model_ResNet50_valid_recall_list.json'))
rec_v = json.load(rec_v)

plt.ion()
plt.style.use("ggplot")
plt.figure()
plt.plot(rec_t, label="train_recall")
plt.plot(rec_v, label="valid_recall")

plt.title("Recall")
plt.xlabel("Epoch #")
plt.ylabel("Recall")
plt.legend(loc="lower right")

In [ ]:
pre_t = open(os.path.join(model_path, 'model_ResNet50_train_precision_list.json'))
pre_t = json.load(pre_t)

pre_v = open(os.path.join(model_path, 'model_ResNet50_valid_precision_list.json'))
pre_v = json.load(pre_v)

plt.ion()

plt.style.use("ggplot")
plt.figure()
plt.plot(pre_t, label="train_precision")
plt.plot(pre_v, label="valid_precision")

plt.title("Precision")
plt.xlabel("Epoch #")
plt.ylabel("Precision")
plt.legend(loc="lower right")

In [ ]:
f1_t = open(os.path.join(model_path, 'model_ResNet50_train_f1_list.json'))
f1_t = json.load(f1_t)

f1_v = open(os.path.join(model_path, 'model_ResNet50_valid_f1_list.json'))
f1_v = json.load(f1_v)

plt.ion()

plt.style.use("ggplot")
plt.figure()
plt.plot(f1_t, label="train_f1")
plt.plot(f1_v, label="valid_f1")

plt.title("F1 score")
plt.xlabel("Epoch #")
plt.ylabel("F1 score")
plt.legend(loc="lower right")

# <center> <font face=timenewroman color=yellow>Test</center>

## <center> <font face=timenewroman color=gree>Prepare model</center>

In [ ]:
model_test = timm.create_model('vgg16', pretrained=True, num_classes=3)

In [ ]:
model_test.load_state_dict(torch.load(model_path + 'model_ResNet50_acc_max.pt'))

In [ ]:
model_test = model_test.cuda()

## <center> <font face=timenewroman color=gree>Start Test</center>

In [ ]:
class_name = ['benign',
              'malignant',
              'normal']

In [ ]:
true_count = 0
all_data   = 0
for class_file in class_name:
  image_path = glob.glob(Dataset + '/test/' + class_file + '/*')
  all_data  += len(glob.glob(Dataset + '/test/' + class_file + '/*'))
  for image in image_path:
    _ ,prediction= app(image, model_test, 'cuda', class_name, True, True)
    print(class_file)
    if class_file == prediction:
      true_count += 1

print('Accuracy of Model = {}'.format(true_count/all_data))